Current workflow

In [3]:
%pylab inline
import lib; lib.reload()

Populating the interactive namespace from numpy and matplotlib


In [4]:
lib.compile_model('vep.stan')

make: `/Users/maedoc/Sources/TVB/VEP/vep' is up to date.
